# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [151]:
import pandas as pd

In [153]:
df = pd.read_csv('previsao_de_renda.csv')

In [155]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.

2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.

3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

In [158]:
import numpy as np
import statsmodels.api as sm
from patsy import dmatrices

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.

In [161]:
# Transformação do log da renda
df['log_renda'] = np.log(df['renda'])


- Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.

In [164]:
# Utilizando Patsy para criar dummies (mantendo a categoria de referência)
# Patsy automaticamente trata variáveis categóricas. Inclui as covariáveis disponíveis.

formula = 'log_renda ~ idade + renda + posse_de_veiculo + sexo + educacao'

In [166]:
# Criação de matrizes design com Patsy (y = dependente, X = preditores)
y, X = dmatrices(formula, data=df, return_type='dataframe')

- Mantenha sempre a categoria mais frequente como casela de referência

In [169]:
# Adicionar uma constante ao modelo (necessário para statsmodels)
X = sm.add_constant(X)

# Ajustando o modelo de regressão (OLS - Ordinary Least Squares)
model = sm.OLS(y, X).fit()

# Resumo do modelo
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:              log_renda   R-squared:                       0.571
Model:                            OLS   Adj. R-squared:                  0.571
Method:                 Least Squares   F-statistic:                     2492.
Date:                Fri, 03 Jan 2025   Prob (F-statistic):               0.00
Time:                        11:21:43   Log-Likelihood:                -13081.
No. Observations:               15000   AIC:                         2.618e+04
Df Residuals:                   14991   BIC:                         2.625e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept 

- Avalie os parâmetros e veja se parecem fazer sentido prático

In [172]:
# Análise dos coeficientes
print("\nCoeficientes interpretáveis:")
for param, value in model.params.items():
    print(f"{param}: {value:.4f}")


Coeficientes interpretáveis:
Intercept: 7.5570
posse_de_veiculo[T.True]: 0.0366
sexo[T.M]: 0.3451
educacao[T.Pós graduação]: 0.0309
educacao[T.Secundário]: -0.0380
educacao[T.Superior completo]: 0.0504
educacao[T.Superior incompleto]: -0.0442
idade: 0.0027
renda: 0.0001


2. Remova a variável menos significante e analise:

In [175]:
# Removendo 'experiencia' do modelo
formula_reduced = 'log_renda ~ idade + renda + posse_de_veiculo + sexo + educacao'

# Criando novamente a matriz design sem a variável 'experiencia'
y_reduced, X_reduced = dmatrices(formula_reduced, data=df, return_type="dataframe")

# Adicionando constante (necessária para statsmodels)
X_reduced = sm.add_constant(X_reduced)

# Ajustando o modelo reduzido
model_reduced = sm.OLS(y_reduced, X_reduced).fit()

# Resumo do modelo reduzido
print(model_reduced.summary())

# Coeficientes do modelo reduzido
print("\nCoeficientes do modelo reduzido:")
for param, value in model_reduced.params.items():
    print(f"{param}: {value:.4f}")

                            OLS Regression Results                            
Dep. Variable:              log_renda   R-squared:                       0.571
Model:                            OLS   Adj. R-squared:                  0.571
Method:                 Least Squares   F-statistic:                     2492.
Date:                Fri, 03 Jan 2025   Prob (F-statistic):               0.00
Time:                        11:21:58   Log-Likelihood:                -13081.
No. Observations:               15000   AIC:                         2.618e+04
Df Residuals:                   14991   BIC:                         2.625e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept 

 - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.

In [178]:
# Comparação entre os modelos
print("\nComparação dos indicadores entre o modelo completo e o reduzido:")
print(f"R² original: {model.rsquared:.4f}, R² ajustado: {model.rsquared_adj:.4f}")
print(f"R² reduzido: {model_reduced.rsquared:.4f}, R² ajustado: {model_reduced.rsquared_adj:.4f}")


Comparação dos indicadores entre o modelo completo e o reduzido:
R² original: 0.5708, R² ajustado: 0.5705
R² reduzido: 0.5708, R² ajustado: 0.5705


- Observe os parâmetros e veja se algum se alterou muito.

In [181]:
# Comparação do F-statistic
print(f"F-statistic original: {model.fvalue:.2f}, p-value: {model.f_pvalue:.4f}")
print(f"F-statistic reduzido: {model_reduced.fvalue:.2f}, p-value: {model_reduced.f_pvalue:.4f}")

F-statistic original: 2491.78, p-value: 0.0000
F-statistic reduzido: 2491.78, p-value: 0.0000


3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor.

In [188]:
# Configurando as variáveis independentes e dependente
X = sm.add_constant(X)  # Adicionando uma constante ao modelo
y = y

# Ajuste do modelo inicial
modelo = sm.OLS(y, X).fit()

# Exibindo o resumo inicial
print("Modelo Inicial:")
print(modelo.summary())

# Remoção de variáveis não significativas
while True:
    p_values = modelo.pvalues  # Obtém os p-valores
    p_values = p_values.drop('const')  # Ignora a constante

    # Obtém o maior p-valor
    max_p_value = p_values.max()

    # Se o maior p-valor for menor que 0.05, encerra o processo
    if max_p_value < 0.05:
        break

    # Remove a variável com maior p-valor
    variavel_remover = p_values.idxmax()
    print(f"Removendo variável '{variavel_remover}' com p-valor de {max_p_value:.4f}")
    X = X.drop(columns=[variavel_remover])

    # Ajusta o modelo novamente
    modelo = sm.OLS(y, X).fit()

# Exibindo o modelo final
print("\nModelo Final:")
print(modelo.summary())

Modelo Inicial:
                            OLS Regression Results                            
Dep. Variable:              log_renda   R-squared:                       0.571
Model:                            OLS   Adj. R-squared:                  0.571
Method:                 Least Squares   F-statistic:                     2492.
Date:                Fri, 03 Jan 2025   Prob (F-statistic):               0.00
Time:                        11:39:01   Log-Likelihood:                -13081.
No. Observations:               15000   AIC:                         2.618e+04
Df Residuals:                   14991   BIC:                         2.625e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------

KeyError: "['const'] not found in axis"